In [50]:
import gzip
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
from sklearn.metrics import mean_absolute_error
from sklearn import tree
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn import svm
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [2]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [66]:
### Helpfulness baseline: similar to the above. Compute the global average helpfulness rate, and the average helpfulness rate for each user

allHelpful = []
wordsReview = []
rating = []
summary = []
userHelpful = defaultdict(list)

for l in readGz("train.json.gz"):
  user,item = l['reviewerID'],l['itemID']
  allHelpful.append(l['helpful'])
  userHelpful[user].append(l['helpful'])
  rating.append(l['rating'])
  wordsReview.append(l['reviewText']) 
  summary.append(l['summary'])

allHelpful = [d for d in allHelpful[:]]
allwordsReview = [d for d in wordsReview[:]]
allRating = [d for d in rating[:]]
allSummary = [d for d in summary[:]]

trainingHelpful = [d for d in allHelpful[0:(len(allHelpful)/2)]]
trainingwordsReview = [d for d in wordsReview[0:len(wordsReview)/2]]
trainingRating = [d for d in rating[0:len(rating)/2]]
trainingSummary = [d for d in summary[0:len(summary)/2]]

validationHelpful = [d for d in allHelpful[(len(allHelpful)/2):len(allHelpful)]]
validationwordsReview = [d for d in wordsReview[len(wordsReview)/2:len(wordsReview)]]
validationRating = [d for d in rating[len(rating)/2:len(rating)]]
validationSummary = [d for d in summary[len(summary)/2:len(summary)]]


In [107]:
#For question 1 on Hw 3 / Compute averagetrainRate to determine alpha = nHelpful/outOf
averagetrainRate = sum([x['nHelpful'] for x in trainingHelpful]) * 1.0 / sum([x['outOf'] for x in trainingHelpful])
print(averagetrainRate)

#For question 2 on HW 3 /Compute averagevalidationRate to determine alpha = nHelpful/outOf
averagevalidationRate = sum([x['nHelpful'] for x in validationHelpful]) * 1.0 / sum([x['outOf'] for x in validationHelpful])
print(averagevalidationRate)

sumValidation = 0
for x in validationHelpful:
    if(x['outOf'] != 0):
        sumValidation += abs((x['nHelpful'] * 1.0/ x['outOf']) - averagevalidationRate)
    else:
        sumValidation += averagevalidationRate
MAE = (sumValidation * 1.0) / len(validationHelpful) 
print(MAE)

#For question 3 on HW 3/ Fit a predictor of the form 
#Function Definition that creates a data entry (row vector) consisting of alpha/words in review/review's rating in stars
def feature(wordsInReview, reviewRating,):
  feat = [1,len(wordsInReview.split()),reviewRating,]
  return feat
#********************************************************************************************************
#For all 200,000 reviews in training data                                                               *
def feature2(wordsInReview, reviewRating,summary):                                                      # 
  feat = [1,len(wordsInReview.split())**10,reviewRating**10]                                            # 
  return feat                                            
XAll = [feature2(x,y,z) for x,y,z in zip(allwordsReview,allRating,allSummary)]
YAll = []

for i in allHelpful:
    if(i['outOf'] != 0):
        YAll.append(i['nHelpful'] * 1.0/i['outOf'])
    else:
        YAll.append(0)
#*************************************************************************************************************
XValidation = [feature(x,y) for x,y in zip(validationwordsReview,validationRating)]
YValidation = []
for i in validationHelpful:
    if(i['outOf'] != 0):
        YValidation.append(i['nHelpful'] * 1.0/i['outOf'])
    else:
        YValidation.append(0)

theta,residuals,rank,s = numpy.linalg.lstsq(XValidation,YValidation)
print(theta)
alpha = 0.224258
beta1 = 0.00135352
beta2 = -0.01353531


yValidationPredictions = []
for d,e in zip(XValidation,validationHelpful):
    yValidationPredictions.append(((d[0] * alpha) + (d[1] * beta1) + (d[2] * beta2))*e['outOf'])
for i,j in zip(YValidation,validationHelpful):
    i = i * j['outOf'] 
print("=========================")
print(mean_absolute_error(YValidation, yValidationPredictions))
print("=========================")

XTraining = [feature(x,y) for x,y in zip(trainingwordsReview,trainingRating)]
YTraining = []
for i in trainingHelpful:
    if(i['outOf'] != 0):
        YTraining.append(i['nHelpful'] * 1.0 / i['outOf'])
    else:
        YTraining.append(0)
        
theta,residuals,rank,s = numpy.linalg.lstsq(XTraining,YTraining)
print(theta)
alpha = 0.21432598
beta1 =  0.00142808
beta2 = -0.01240441
yTrainingPredictions = []
for d,e in zip(XTraining,trainingHelpful):
    yTrainingPredictions.append(((d[0] * alpha) + (d[1] * beta1) + (d[2] * beta2)) * e['outOf']) 
for i,j in zip(YTraining,trainingHelpful):
    i = i * j['outOf']
print("=========================")
print(mean_absolute_error(YTraining, yTrainingPredictions))
print("=========================")



#averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / (sum([x['outOf'] for x in allHelpful]))
#print(averageRate)
#userRate = {}
#for u in userHelpful:
#  totalU = sum([x['outOf'] for x in userHelpful[u]])
#  if totalU > 0:
#    userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / totalU
#  else:
#    userRate[u] = averageRate



0.85040619252
0.853527624877
0.664070799349
[ 0.224258    0.00135352 -0.01353531]
0.409520148467
[ 0.21432598  0.00142808 -0.01240441]
0.417198260867


In [101]:
print(len(YAll))

200000


In [108]:

regressor = RandomForestRegressor(n_estimators=1000)
regressor.fit(XAll, YAll)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [111]:

count = 0 
userHelpful = defaultdict(list)
rating = []
wordsReview = []
testHelpful = []
summary = []
for l in readGz("test_Helpful.json.gz"):
  count += 1
  user,item = l['reviewerID'],l['itemID']
  userHelpful[user].append(item)
  rating.append(l['rating'])
  wordsReview.append(l['reviewText']) 
  testHelpful.append(l)
  summary.append(l['summary'])

testwordsReview = [d for d in wordsReview[:]]
testRating = [d for d in rating[:]]
testSummary = [d for d in summary[:]]

XTest = [feature2(x,y,z) for x,y,z in zip(testwordsReview, testRating, testSummary)]

alpha = 0.224258
beta1 = 0.00135352
beta2 = -0.01353531

yTestPredictions = []
for d in XTest:
    yTestPredictions.append((d[0] * alpha) + (d[1] * beta1) + (d[2] * beta2))

    allUsersTestSet = []
for a,b in zip(readGz("test_Helpful.json.gz"),yTestPredictions):
     allUsersTestSet.append([a['reviewerID'],a['itemID'],b])
    

hold = regressor.predict(XTest)
print(len(hold))




14000


In [116]:
predictions = open("predictions_Helpful.txt", 'w')
counter = 0
for l in open("pairs_Helpful.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i,outOf = l.strip().split('-')
  outOf = int(outOf)
    
  predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*hold[counter]) + '\n')  #Need to multiply by outOf
  counter += 1
  #for x in allUsersTestSet:
  #   if(x[0] == u) and (x[1] == i):
  #          predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*x[2]) + '\n')
  #          break

predictions.close()

In [8]:
#Question 5 on HW # 3 (Category Prediction)
### Category prediction baseline: Just consider some of the most common words from each category
allCategoryReviews = []
userCategory = defaultdict(list)

for l in readGz("train.json.gz"):
  user,item = l['reviewerID'],l['itemID']
  allCategoryReviews.append(l)
  if(user not in userCategory):
     userCategory[user].append([0,0,0,0,0])


trainingCategory = [d for d in allHelpful[0:(len(allCategoryReviews)/2)]]

validationCategory = [d for d in allHelpful[(len(allCategoryReviews)/2):len(allCategoryReviews)]]


In [9]:

counter = 0

for d in readGz("train.json.gz"):
    if(counter >= 100000):
        break
    else:
        listxs = userCategory.get(d['reviewerID'])
        if(d['categoryID'] == 0):
            listxs[0][0] = listxs[0][0] + 1
        elif(d['categoryID'] == 1):
            listxs[0][1] = listxs[0][1] + 1
        elif(d['categoryID'] == 2):
            listxs[0][2] = listxs[0][2] + 1
        elif(d['categoryID'] == 3):
            listxs[0][3] = listxs[0][3] + 1
        elif(d['categoryID'] == 4):
            listxs[0][4] = listxs[0][4] + 1
        userCategory[d['reviewerID']] = listxs
        counter += 1



In [10]:
print(userCategory['U127983630'][0])
counter = 0
categoricalReview = []
ActualCategory = []
for d in readGz("train.json.gz"):
    if(counter < 100000):
        counter += 1
        continue
    else:
        category1 = userCategory[d['reviewerID']][0][0] 
        category2 = userCategory[d['reviewerID']][0][1]
        category3 = userCategory[d['reviewerID']][0][2]
        category4 = userCategory[d['reviewerID']][0][3]
        category5 = userCategory[d['reviewerID']][0][4]
        catDict = {0 : category1, 1 : category2, 2 : category3, 3 : category4, 4 : category5}
        categoricalReview.append(max(catDict, key=lambda k: catDict[k]))

        ActualCategory.append(d['categoryID'])

        

[1, 0, 0, 0, 2]


In [11]:
print(mean_absolute_error(categoricalReview, ActualCategory))

0.2749


In [12]:
#Question 6

### Ignore capitalization and remove punctuation

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    ### Just take the most popular words...
    r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
    counter += 1
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:500]]
sumTotalWords = 0;
for d in counts[:500]:
    sumTotalWords += d[0]

fW = {}
for d in counts[:500]:
    fW[d[1]] = d[0] * 1.0 / sumTotalWords
#frequencyWords = {}
#for d in words:
    

In [13]:
wordCountWomen = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in counts[:500]:
    wordCountWomen[d[1]] = 0 
print(wordCountWomen)
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    counter += 1
    if(d['categoryID'] == 0):
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if(wordCountWomen.has_key(w)):
                wordCountWomen[w] += 1
                
wordCountWomenT = [(wordCountWomen[w], w) for w in wordCountWomen]
wordCountWomenT.sort()
wordCountWomenT.reverse()

sumTotalWords2 = 0;
for d in wordCountWomenT:
    sumTotalWords2 += d[0]
    
fWWomen = {}
for d in wordCountWomenT:
    fWWomen[d[1]] = d[0] * 1.0 / sumTotalWords2

frequencyWordWomenCategory = defaultdict(int)
for d in counts[:500]:
    frequencyWordWomenCategory[d[1]] = fWWomen[d[1]] - fW[d[1]]

womenCounts = [(frequencyWordWomenCategory[w], w) for w in frequencyWordWomenCategory]
womenCounts = sorted(womenCounts, key=lambda x: x[0])
print(womenCounts)
catDict = {'Women': 0, 'Men': 1, 'Girls': 2, 'Boys': 3, 'Baby': 4}









defaultdict(<type 'int'>, {'all': 0, 'chain': 0, 'go': 0, 'everything': 0, 'to': 0, 'under': 0, 'worth': 0, 'sizing': 0, '7': 0, 'outside': 0, 'very': 0, 'every': 0, 'fall': 0, 'room': 0, 'durable': 0, 'did': 0, 'large': 0, 'item': 0, 'small': 0, 'havent': 0, 'clothes': 0, 'smaller': 0, 'says': 0, 'boots': 0, 'second': 0, 'design': 0, 'shirts': 0, 'uncomfortable': 0, 'casual': 0, 'blue': 0, 'what': 0, 'pleased': 0, 'wouldnt': 0, 'goes': 0, 'new': 0, 'ever': 0, 'can': 0, 'body': 0, 'toes': 0, 'never': 0, 'here': 0, 'water': 0, 'others': 0, 'sexy': 0, 'box': 0, 'great': 0, 'daughter': 0, 'my': 0, 'larger': 0, 'arent': 0, 'usually': 0, 'narrow': 0, 'makes': 0, 'thats': 0, 'love': 0, 'extra': 0, 'highly': 0, 'snug': 0, 'bra': 0, 'use': 0, 'from': 0, 'would': 0, 'two': 0, 'next': 0, 'few': 0, 'recommend': 0, 'type': 0, 'tell': 0, 'more': 0, 'warm': 0, 'excellent': 0, 'hold': 0, 'glad': 0, 'me': 0, 'wore': 0, 'this': 0, 'work': 0, 'worn': 0, 'reviews': 0, 'wallet': 0, 'thin': 0, 'didnt': 0, 

In [14]:
wordCountMen = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in counts[:500]:
    wordCountMen[d[1]] = 0 
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    counter += 1
    if(d['categoryID'] == 1):
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if(wordCountMen.has_key(w)):
                wordCountMen[w] += 1
                
wordCountMenT = [(wordCountMen[w], w) for w in wordCountMen]
wordCountMenT.sort()
wordCountMenT.reverse()

sumTotalWords3 = 0;
for d in wordCountMenT:
    sumTotalWords3 += d[0]
    
fWMen = {}
for d in wordCountMenT:
    fWMen[d[1]] = d[0] * 1.0 / sumTotalWords3

frequencyWordMenCategory = defaultdict(int)
for d in counts[:500]:
    frequencyWordMenCategory[d[1]] = fWMen[d[1]] - fW[d[1]]

menCounts = [(frequencyWordMenCategory[w], w) for w in frequencyWordMenCategory]
menCounts = sorted(menCounts, key=lambda x: x[0])
print(menCounts)



[(-0.007997242946658212, 'i'), (-0.0038223204422464657, 'it'), (-0.002332963293780616, 'love'), (-0.0017208595291963464, 'bra'), (-0.0015779990897627529, 'so'), (-0.0015384736424706915, 'cute'), (-0.0013496191373385613, 'size'), (-0.0013067561367297786, 'wear'), (-0.001252993866617115, 'she'), (-0.001212793454802111, 'but'), (-0.0011527194118167465, 'was'), (-0.00092994616757517, 'dress'), (-0.0009188575418925065, 'am'), (-0.0008325650322960963, 'small'), (-0.0007859564813043131, 'her'), (-0.0007847419423473611, 'top'), (-0.0007737644737451932, 'color'), (-0.0007393741195582885, 'ordered'), (-0.000722801367130415, 'them'), (-0.0006541817653842826, 'too'), (-0.0006353885453598204, 'this'), (-0.0006192222198308543, 'pretty'), (-0.000598697323171729, 'beautiful'), (-0.0005973233872021743, 'im'), (-0.0005874075836940643, 'would'), (-0.0005295033573775626, 'little'), (-0.0005194280427789611, 'because'), (-0.0005011718362175693, 'colors'), (-0.0004962177439919181, 'bras'), (-0.00047828756496

In [15]:
wordCountGirls = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in counts[:500]:
    wordCountGirls[d[1]] = 0 
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    counter += 1
    if(d['categoryID'] == 2):
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if(wordCountGirls.has_key(w)):
                wordCountGirls[w] += 1
                
wordCountGirlsT = [(wordCountGirls[w], w) for w in wordCountGirls]
wordCountGirlsT.sort()
wordCountGirlsT.reverse()

sumTotalWords4 = 0;
for d in wordCountGirlsT:
    sumTotalWords4 += d[0]
    
fWGirls = {}
for d in wordCountGirlsT:
    fWGirls[d[1]] = d[0] * 1.0 / sumTotalWords4

frequencyWordGirlsCategory = defaultdict(int)
for d in counts[:500]:
    frequencyWordGirlsCategory[d[1]] = fWGirls[d[1]] - fW[d[1]]

girlsCounts = [(frequencyWordGirlsCategory[w], w) for w in frequencyWordGirlsCategory]
girlsCounts = sorted(girlsCounts, key=lambda x: x[0])
print(girlsCounts)



[(-0.01112654058841095, 'i'), (-0.002849959298239311, 'in'), (-0.0027837641045285032, 'comfortable'), (-0.0026117440147816343, 'have'), (-0.0020803568325772044, 'you'), (-0.002029791449250352, 'me'), (-0.0017890995318877306, 'a'), (-0.0016857461748908988, 'wear'), (-0.0016820386301751866, 'bra'), (-0.0016600005064384867, 'not'), (-0.0016331073736411943, 'of'), (-0.0015869946536746225, 'im'), (-0.0014759328173267281, 'that'), (-0.0014451210068290856, 'shoe'), (-0.0013878826526485283, 'am'), (-0.0013722699505175247, 'these'), (-0.0013434445822436895, 'pair'), (-0.0012205629285745867, 'shoes'), (-0.0011696880704215634, 'if'), (-0.0011042095690659143, 'the'), (-0.0010873229974409362, 'he'), (-0.0010837707433576354, 'but'), (-0.0010723386343843286, 'your'), (-0.0010331963117241995, 'feet'), (-0.0010270286290979484, 'are'), (-0.000982584475467176, 'love'), (-0.0009671355000166229, 'feel'), (-0.0009081379914758304, 'shirt'), (-0.0008892059471571251, 'support'), (-0.000888541874050301, 'good')

In [16]:
wordCountBoys = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in counts[:500]:
    wordCountBoys[d[1]] = 0 
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    counter += 1
    if(d['categoryID'] == 3):
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if(wordCountBoys.has_key(w)):
                wordCountBoys[w] += 1
                
wordCountBoysT = [(wordCountBoys[w], w) for w in wordCountBoys]
wordCountBoysT.sort()
wordCountBoysT.reverse()

sumTotalWords5 = 0;
for d in wordCountBoysT:
    sumTotalWords5 += d[0]
    
fWBoys = {}
for d in wordCountBoysT:
    fWBoys[d[1]] = d[0] * 1.0 / sumTotalWords5

frequencyWordBoysCategory = defaultdict(int)
for d in counts[:500]:
    frequencyWordBoysCategory[d[1]] = fWBoys[d[1]] - fW[d[1]]

boysCounts = [(frequencyWordBoysCategory[w], w) for w in frequencyWordBoysCategory]
boysCounts = sorted(boysCounts, key=lambda x: x[0])
print(boysCounts)

[(-0.019991919773250336, 'i'), (-0.0033665917806088904, 'have'), (-0.0032948529991819984, 'it'), (-0.0031357562318454794, 'you'), (-0.003008042819446494, 'me'), (-0.002771910837328088, 'comfortable'), (-0.002762053607517882, 'shoes'), (-0.0024767710385153185, 'a'), (-0.0022068053119916565, 'wear'), (-0.002185403745368672, 'this'), (-0.002000348843634185, 'like'), (-0.0017906032975634535, 'am'), (-0.001758595950841394, 'im'), (-0.0017233021181778275, 'bra'), (-0.0016065102330187712, 'love'), (-0.0015980295920834688, 'shoe'), (-0.0015974998305268017, 'pair'), (-0.0014417050088229512, 'its'), (-0.0014017464644344969, 'work'), (-0.0013646543798195832, 'or'), (-0.0013325391619656989, 'feet'), (-0.0013196291944708692, 'to'), (-0.0013113496545622948, 'that'), (-0.0012666182090293678, 'look'), (-0.001194876744891363, 'in'), (-0.0011741995841572266, 'color'), (-0.001164459637702583, 'of'), (-0.0011591584300312003, 'but'), (-0.0011082766760246608, 'watch'), (-0.0010646109374815344, 'top'), (-0.0

In [17]:
wordCountBaby = defaultdict(int)
punctuation = set(string.punctuation)
counter = 0
for d in counts[:500]:
    wordCountBaby[d[1]] = 0 
print(wordCountBaby)
for d in readGz("train.json.gz"):
  if(counter >= 100000):
    break
  else:
    counter += 1
    if(d['categoryID'] == 4):
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if(wordCountBaby.has_key(w)):
                wordCountBaby[w] += 1
                
wordCountBabyT = [(wordCountBaby[w], w) for w in wordCountBaby]
wordCountBabyT.sort()
wordCountBabyT.reverse()

sumTotalWords6 = 0;
for d in wordCountBabyT:
    sumTotalWords6 += d[0]
    
fWBaby = {}
for d in wordCountBabyT:
    fWBaby[d[1]] = d[0] * 1.0 / sumTotalWords6

frequencyWordBabyCategory = defaultdict(int)
for d in counts[:500]:
    frequencyWordBabyCategory[d[1]] = fWBaby[d[1]] - fW[d[1]]

babyCounts = [(frequencyWordBabyCategory[w], w) for w in frequencyWordBabyCategory]
babyCounts = sorted(babyCounts, key=lambda x: x[0])


defaultdict(<type 'int'>, {'all': 0, 'chain': 0, 'go': 0, 'everything': 0, 'to': 0, 'under': 0, 'worth': 0, 'sizing': 0, '7': 0, 'outside': 0, 'very': 0, 'every': 0, 'fall': 0, 'room': 0, 'durable': 0, 'did': 0, 'large': 0, 'item': 0, 'small': 0, 'havent': 0, 'clothes': 0, 'smaller': 0, 'says': 0, 'boots': 0, 'second': 0, 'design': 0, 'shirts': 0, 'uncomfortable': 0, 'casual': 0, 'blue': 0, 'what': 0, 'pleased': 0, 'wouldnt': 0, 'goes': 0, 'new': 0, 'ever': 0, 'can': 0, 'body': 0, 'toes': 0, 'never': 0, 'here': 0, 'water': 0, 'others': 0, 'sexy': 0, 'box': 0, 'great': 0, 'daughter': 0, 'my': 0, 'larger': 0, 'arent': 0, 'usually': 0, 'narrow': 0, 'makes': 0, 'thats': 0, 'love': 0, 'extra': 0, 'highly': 0, 'snug': 0, 'bra': 0, 'use': 0, 'from': 0, 'would': 0, 'two': 0, 'next': 0, 'few': 0, 'recommend': 0, 'type': 0, 'tell': 0, 'more': 0, 'warm': 0, 'excellent': 0, 'hold': 0, 'glad': 0, 'me': 0, 'wore': 0, 'this': 0, 'work': 0, 'worn': 0, 'reviews': 0, 'wallet': 0, 'thin': 0, 'didnt': 0, 

In [18]:
xSVM = []
ySVM = []
counter = 0
for d in readGz("train.json.gz"):
    if(counter >= 5000):
        break
    else:
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 0):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xSVM.append(svmWordCount.values())
            ySVM.append(0)
        elif(d['categoryID'] == 1):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xSVM.append(svmWordCount.values())
            ySVM.append(1)
        counter += 1
        

          
       

In [19]:
clf = svm.SVC(C=100)
clf.fit(xSVM, ySVM)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
xValidationSVM = []
yValidationSVM = []
counter = 0
for d in readGz("train.json.gz"):
    if(counter < 5000):
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 2):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(2)
        elif(d['categoryID'] == 3):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(3)
        elif(d['categoryID'] == 4):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(4)
        counter += 1
    else:
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 0):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(0)
        elif(d['categoryID'] == 1):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(1)
        elif(d['categoryID'] == 2):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(2)
        elif(d['categoryID'] == 3):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(3)
        elif(d['categoryID'] == 4):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xValidationSVM.append(svmWordCount.values())
            yValidationSVM.append(4)

        

In [21]:
print(len(xSVM))
print(len(xValidationSVM))
print(len(yValidationSVM))

4818
195182
195182


In [22]:
test_predictions = clf.predict(xValidationSVM)
test_labels = [a > 0 for a in test_predictions]
test_correct = [(a == b) for a,b in zip(test_labels, yValidationSVM)]

In [23]:
print(sum(test_correct) * 1.0 / len(test_correct) * 100)

76.0608047873


In [24]:
#Question 8
xtrainingSVM8 = []
ytrainingSVM8 = []
counter = 0

for d in readGz("train.json.gz"):
    if(counter < 20000):
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 0):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVM8.append(svmWordCount.values())
            ytrainingSVM8.append(0)
        elif(d['categoryID'] == 1):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVM8.append(svmWordCount.values())
            ytrainingSVM8.append(1)
        elif(d['categoryID'] == 2):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVM8.append(svmWordCount.values())
            ytrainingSVM8.append(2)
        elif(d['categoryID'] == 3):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVM8.append(svmWordCount.values())
            ytrainingSVM8.append(3)
        elif(d['categoryID'] == 4):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVM8.append(svmWordCount.values())
            ytrainingSVM8.append(4)
        counter += 1

In [25]:
xvalidationSVM8 = []
yvalidationSVM8 = []
counter = 0

for d in readGz("train.json.gz"):
    if(counter < 20000):
        counter += 1
    elif(counter >= 20000 and counter < 40000):
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 0):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xvalidationSVM8.append(svmWordCount.values())
            yvalidationSVM8.append(0)
        elif(d['categoryID'] == 1):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xvalidationSVM8.append(svmWordCount.values())
            yvalidationSVM8.append(1)
        elif(d['categoryID'] == 2):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xvalidationSVM8.append(svmWordCount.values())
            yvalidationSVM8.append(2)
        elif(d['categoryID'] == 3):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xvalidationSVM8.append(svmWordCount.values())
            yvalidationSVM8.append(3)
        elif(d['categoryID'] == 4):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xvalidationSVM8.append(svmWordCount.values())
            yvalidationSVM8.append(4)
        counter += 1

In [26]:
clf = svm.LinearSVC(C= 100)
clf.fit(xtrainingSVM8, ytrainingSVM8)

LinearSVC(C=100, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [27]:
test_decision = clf.decision_function(xvalidationSVM8)

In [28]:

resultvalidationSVM = []
for d in test_decision:
    d = list(d)
    m = d.index(max(d))
    if(m == 0):
        resultvalidationSVM.append(0)
    elif(m == 1):
         resultvalidationSVM.append(1)
    elif(m == 2):
         resultvalidationSVM.append(2)
    elif(m == 3):
         resultvalidationSVM.append(3)
    elif(m == 4):
         resultvalidationSVM.append(4)


In [29]:
print(resultvalidationSVM)

[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 3, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 3, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 

In [30]:
test_correct = [(a == b) for a,b in zip(resultvalidationSVM, yvalidationSVM8)]
#print(yvalidationSVM8)
print(sum(test_correct) * 1.0 / len(test_correct) * 100)

74.08


In [31]:
#Testing on test data
xtrainingSVMF = []
ytrainingSVMF = []

for d in readGz("train.json.gz"):
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        if(d['categoryID'] == 0):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVMF.append(svmWordCount.values())
            ytrainingSVMF.append(0)
        elif(d['categoryID'] == 1):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVMF.append(svmWordCount.values())
            ytrainingSVMF.append(1)
        elif(d['categoryID'] == 2):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVMF.append(svmWordCount.values())
            ytrainingSVMF.append(2)
        elif(d['categoryID'] == 3):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVMF.append(svmWordCount.values())
            ytrainingSVMF.append(3)
        elif(d['categoryID'] == 4):
            r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
            for w in r.split():
                if(svmWordCount.has_key(w)):
                    svmWordCount[w] = 1
            xtrainingSVMF.append(svmWordCount.values())
            ytrainingSVMF.append(4)
    

In [32]:
clf = svm.LinearSVC(C= 0.000000000000000000000000000000000000000000000001)
clf.fit(xtrainingSVMF, ytrainingSVMF)


LinearSVC(C=1e-48, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [33]:
xtestSVMF = []

for d in readGz("test_Category.json.gz"):
        svmWordCount = defaultdict(int)
        for e in words:
            svmWordCount[e] = 0
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
             if(svmWordCount.has_key(w)):
                svmWordCount[w] = 1
        xtestSVMF.append(svmWordCount.values())
        
    

In [34]:
test_decision = clf.decision_function(xvalidationSVM8)

In [35]:
resulttestSVM = []
for d in test_decision:
    d = list(d)
    m = d.index(max(d))
    if(m == 0):
        resulttestSVM.append(0)
    elif(m == 1):
         resulttestSVM.append(1)
    elif(m == 2):
         resulttestSVM.append(2)
    elif(m == 3):
         resulttestSVM.append(3)
    elif(m == 4):
         resulttestSVM.append(4)
            

In [36]:
print(resulttestSVM)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [37]:
counter = 0
predictions = open("predictions_Category.txt", 'w')
for l in open("pairs_Category.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    predictions.write(u + '-' + i +',' + str(resulttestSVM[counter]) + '\n')
    counter += 1


predictions.close()